This file is used for generating *EEG Images* from Spectral Power Vectors (SPV) features using the method specified [here](https://arxiv.org/abs/1511.06448). SPV features are obtained using `eeg_spectral.ipynb`. Refer Section III-E in the paper for the process involved in transforming SPV features to *EEG Images*. The highlighted part in the following image indicate the pipeline of the current file.

Note: Thanks to Pouya Bashivan. Snippets in this code are based on [https://github.com/pbashivan/EEGLearn](https://github.com/pbashivan/EEGLearn)


![EEG features to EEG Images](images/eeg_images.png)

# Imports

In [1]:
import math as m
import numpy as np
from scipy.io import loadmat, savemat
from scipy.interpolate import griddata
from sklearn.preprocessing import scale

# EEG signal to Images

In [2]:

def cart2sph(x, y, z):
    """
    Transform Cartesian coordinates to spherical
    :param x: X coordinate
    :param y: Y coordinate
    :param z: Z coordinate
    :return: radius, elevation, azimuth
    """
    x2_y2 = x**2 + y**2
    r = m.sqrt(x2_y2 + z**2)                    # r
    elev = m.atan2(z, m.sqrt(x2_y2))            # Elevation
    az = m.atan2(y, x)                          # Azimuth
    return r, elev, az


def pol2cart(theta, rho):
    """
    Transform polar coordinates to Cartesian
    :param theta: angle value
    :param rho: radius value
    :return: X, Y
    """
    return rho * m.cos(theta), rho * m.sin(theta)


def azim_proj(pos):
    """
    Computes the Azimuthal Equidistant Projection of input point in 3D Cartesian Coordinates.
    Imagine a plane being placed against (tangent to) a globe. If
    a light source inside the globe projects the graticule onto
    the plane the result would be a planar, or azimuthal, map
    projection.

    :param pos: position in 3D Cartesian coordinates
    :return: projected coordinates using Azimuthal Equidistant Projection
    """
    [r, elev, az] = cart2sph(pos[0], pos[1], pos[2])
    return pol2cart(az, m.pi / 2 - elev)

def augment_EEG(data, stdMult, pca=False, n_components=2):
    """
    Augment data by adding normal noise to each feature.
    :param data: EEG feature data as a matrix (n_samples x n_features)
    :param stdMult: Multiplier for std of added noise
    :param pca: if True will perform PCA on data and add noise proportional to PCA components.
    :param n_components: Number of components to consider when using PCA.
    :return: Augmented data as a matrix (n_samples x n_features)
    """
    augData = np.zeros(data.shape)
    if pca:
        pca = PCA(n_components=n_components)
        pca.fit(data)
        components = pca.components_
        variances = pca.explained_variance_ratio_
        coeffs = np.random.normal(scale=stdMult, size=pca.n_components) * variances
        for s, sample in enumerate(data):
            augData[s, :] = sample + (components * coeffs.reshape((n_components, -1))).sum(axis=0)
    else:
        # Add Gaussian noise with std determined by weighted std of each feature
        for f, feat in enumerate(data.transpose()):
            augData[:, f] = feat + np.random.normal(scale=stdMult*np.std(feat), size=feat.size)
    return augData
    
def gen_images(locs, features, n_gridpoints, normalize=True,
               augment=False, pca=False, std_mult=0.1, n_components=2, edgeless=False):
    """
    Generates EEG images given electrode locations in 2D space and multiple feature values for each electrode

    :param locs: An array with shape [n_electrodes, 2] containing X, Y
                        coordinates for each electrode.
    :param features: Feature matrix as [n_samples, n_features]
                                Features are as columns.
                                Features corresponding to each frequency band are concatenated.
                                (alpha1, alpha2, ..., beta1, beta2,...)
    :param n_gridpoints: Number of pixels in the output images
    :param normalize:   Flag for whether to normalize each band over all samples
    :param augment:     Flag for generating augmented images
    :param pca:         Flag for PCA based data augmentation
    :param std_mult     Multiplier for std of added noise
    :param n_components: Number of components in PCA to retain for augmentation
    :param edgeless:    If True generates edgeless images by adding artificial channels
                        at four corners of the image with value = 0 (default=False).
    :return:            Tensor of size [samples, colors, W, H] containing generated
                        images.
    """
    feat_array_temp = []
    nElectrodes = locs.shape[0]     # Number of electrodes

    # Test whether the feature vector length is divisible by number of electrodes
    assert features.shape[1] % nElectrodes == 0
    n_colors = features.shape[1] / nElectrodes
    for c in range(int(n_colors)):
        feat_array_temp.append(features[:, c * nElectrodes : nElectrodes * (c+1)])
    #print(feat_array_temp)
    if augment:
        if pca:
            for c in range(int(n_colors)):
                feat_array_temp[c] = augment_EEG(feat_array_temp[c], std_mult, pca=True, n_components=n_components)
        else:
            for c in range(int(n_colors)):
                feat_array_temp[c] = augment_EEG(feat_array_temp[c], std_mult, pca=False, n_components=n_components)
    n_samples = features.shape[0]
    #print(n_samples)
    # Interpolate the values
    grid_x, grid_y = np.mgrid[
                     min(locs[:, 0]):max(locs[:, 0]):n_gridpoints*1j,
                     min(locs[:, 1]):max(locs[:, 1]):n_gridpoints*1j
                     ]
    temp_interp = []
    for c in range(int(n_colors)):
        temp_interp.append(np.zeros([n_samples, n_gridpoints, n_gridpoints]))
    #print(temp_interp)
    # Generate edgeless images
    if edgeless:
        min_x, min_y = np.min(locs, axis=0)
        max_x, max_y = np.max(locs, axis=0)
        locs = np.append(locs, np.array([[min_x, min_y], [min_x, max_y], [max_x, min_y], [max_x, max_y]]), axis=0)
        for c in range(n_colors):
            feat_array_temp[c] = np.append(feat_array_temp[c], np.zeros((n_samples, 4)), axis=1)

    # Interpolating
    for i in range(n_samples):
      for c in range(int(n_colors)):
        temp_interp[c][i, :, :] = griddata(locs, feat_array_temp[c][i, :], (grid_x, grid_y),
                                           method='cubic', fill_value=np.nan)
        #print('Interpolating {0}/{1}\n'.format(i + 1, n_samples), end='\n')
    #print(temp_interp)

    # Normalizing
    del feat_array_temp
    
    for c in range(int(n_colors)):
        if normalize:
            temp_interp[c][~np.isnan(temp_interp[c])] = temp_interp[c][~np.isnan(temp_interp[c])].astype(float)
            temp_interp[c][~np.isnan(temp_interp[c])] = \
                scale(temp_interp[c][~np.isnan(temp_interp[c])])
        temp_interp[c] = np.nan_to_num(temp_interp[c])
    return np.swapaxes(np.asarray(temp_interp), 0, 1)     # swap axes to have [samples, colors, W, H]


def gen_images_3d(locs, features, n_gridpoints, normalize=True,
               augment=False, pca=False, std_mult=0.1, n_components=2, edgeless=False):
  """
  EEG to images for 3D CNNs.
  :features: Feature matrix of the shape [n_samples, length, n_features] (eg: (14800, 5, 42))
  """
  n_samples = features.shape[0]
  nElectrodes = locs.shape[0]     # Number of electrodes
  # Test whether the feature vector length is divisible by number of electrodes
  assert features.shape[2] % nElectrodes == 0
  n_colors = features.shape[2] / nElectrodes

  images_mat = np.empty((0, int(features.shape[1]),int(n_colors), n_gridpoints, n_gridpoints)) #shape (0, 5, 3, 32, 32)
  for sample in range(n_samples):
    feature_split = features[sample, :, :] #shape (5, 42)
    temp_images_mat = gen_images(locs, feature_split, n_gridpoints, normalize=True, augment=False, 
                                 pca=False, std_mult=0.1, n_components =2, edgeless=False)
    temp_images_mat = np.expand_dims(temp_images_mat, axis=0) #shape (1, 5, 3, 32, 32)
    images_mat = np.concatenate((images_mat, temp_images_mat), axis=0) # shape (samples, 5, 3, 32, 32)
    #print(images_mat.shape)
  return images_mat


# Main loop

In [ ]:
if __name__ == '__main__':
 
    # Load electrode locations
    print('Loading data...')
    locs = loadmat('../data/chans.mat')
    locs_3d = locs['x']
    locs_2d = []
    # Convert to 2D
    for e in locs_3d:
        locs_2d.append(azim_proj(e))
    
    #matrices = ['full_feat']
    matrices = ['full_feat', 'pd_feat', 'nc_feat', 'E1_feat', 'E2_feat', 'E3_feat', 'E4_feat', 'E5_feat', 'E6_feat']
    matContent = loadmat('../data/spectral_feat_tensor_full_with_full_labels.mat')
    filename = '../data/spectral_feat_tensor_full_with_full_labels_images_128_new.pkl'
    n_grid = 128

    for matrix in matrices:
        features = matContent[matrix]
        del matContent[matrix]
        print('Generating images for', matrix)
        print(matrix, 'shape:', features.shape)
        images = gen_images(np.array(locs_2d), features, n_grid, normalize = True)
        print('Image size:', np.shape(images))
        matContent[matrix+'_img'] = images
        del images
        print('\n')
    print("saving...")
    #savemat(filename, matContent)
    print("Save file done..")

In [7]:

import pickle
filename = '../data/spectral_feat_tensor_full_with_full_labels_images_128_new.pkl'
try:
    mat_file = open(filename, 'wb')
    #pickle.dump(matContent, mat_file, pickle.HIGHEST_PROTOCOL)
    pickle.dump(matContent, mat_file, protocol=4)
    mat_file.close()
except:
    print("Something went wrong")


# Archive

In [5]:
'''
print('Generating images...')
images = gen_images(np.array(locs_2d),features, 128, normalize=True)
print('\n')
#print(images)
print(np.shape(images))
'''

"\nprint('Generating images...')\nimages = gen_images(np.array(locs_2d),features, 128, normalize=True)\nprint('\n')\n#print(images)\nprint(np.shape(images))\n"

In [6]:
'''
import PIL
import PIL.Image
import tensorflow as tf
import matplotlib.pyplot as plt
images1 = np.swapaxes(images,2,4)
print(images1.shape)
images1 = np.swapaxes(images1,2,3)
print(images1.shape)
plt.imshow(images1[1995][3])
'''

'\nimport PIL\nimport PIL.Image\nimport tensorflow as tf\nimport matplotlib.pyplot as plt\nimages1 = np.swapaxes(images,2,4)\nprint(images1.shape)\nimages1 = np.swapaxes(images1,2,3)\nprint(images1.shape)\nplt.imshow(images1[1995][3])\n'